In [1]:
import pandas as pd
import panel as pn
import numpy as np
import os
from sqlalchemy import create_engine, text
from datetime import date, datetime, timedelta
import calendar
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set

pn.extension('tabulator')
pd.options.display.max_rows = 11

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

today = date.today()
today

datetime.date(2025, 8, 12)

In [2]:
num_business_days = BDay(2)
yesterday = today - num_business_days
yesterday = yesterday.date()
today, yesterday

(datetime.date(2025, 8, 12), datetime.date(2025, 8, 8))

### Restart and Run All Cells

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}")

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Ad hoc
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
sqlUpd = text("UPDATE buy B SET dividend = (SELECT dividend FROM dividend D WHERE B.name = D.name)")
rp = const.execute(sqlUpd)
rp.rowcount

28

In [7]:
cols = 'name period shares mkt_price cost_amt base_amt pct'.split()
colt = 'name shares unit_cost cost_amt price market_amt mkt_pct div_rnk amt_rnk div_amt'.split()
colu = 'name shares unit_cost cost_amt dividend div_amt cost_pct period'.split()
colv = 'name shares unit_cost cost_amt mkt_price market_amt dividend div_amt cost_pct mkt_pct profit pft_pct'.split()
colw = 'name cost_amt market_amt div_amt cost_pct mkt_pct pft_pct'.split()
colx = 'name shares unit_cost cost_amt mkt_price market_amt dividend div_amt cost_pct mkt_pct profit pft_pct period'.split()

In [8]:
format_dict = {
    'shares':'{:,}',  
    'mkt_price':'฿{:.2f}','unit_cost':'฿{:.2f}',
    'cost_amt':'฿{:,.2f}','div_amt':'฿{:,.2f}','market_amt':'฿{:,.2f}','profit':'฿{:,.2f}','base_amt':'฿{:,.2f}',
    'dividend':'฿{:.4f}',    
    'pct':'{:,.2f}%','cost_pct':'{:,.2f}%','mkt_pct':'{:,.2f}%','pft_pct':'{:,.2f}%',
}

### Process portfolio (table buy in mysql stock)

In [10]:
sql = """
SELECT name, volbuy, price AS unit_cost, dividend, period
FROM buy
WHERE active = 1"""
print(sql)


SELECT name, volbuy, price AS unit_cost, dividend, period
FROM buy
WHERE active = 1


In [11]:
buy = pd.read_sql(sql, const)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy['cost_amt'] = round(buy['shares'] * buy['unit_cost'], 2)
buy['cost_pct'] = round(buy['dividend'] / buy['unit_cost'] * 100, 2)
buy['div_amt'] = round(buy['shares'] * buy['dividend'], 2)
buy[colu].sort_values('cost_pct',ascending=False).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
17,GVREIT,"69,000",฿7.75,"฿534,750.00",฿0.7989,"฿55,124.10",10.31%,2
19,AIMIRT,"15,000",฿10.75,"฿161,250.00",฿0.8500,"฿12,750.00",7.91%,2
27,TVO,"8,000",฿24.00,"฿192,000.00",฿1.7300,"฿13,840.00",7.21%,3
5,DIF,"40,000",฿13.10,"฿524,000.00",฿0.8888,"฿35,552.00",6.78%,2
3,PTT,"2,500",฿32.00,"฿80,000.00",฿2.1000,"฿5,250.00",6.56%,2
10,TFFIF,"20,000",฿7.20,"฿144,000.00",฿0.4596,"฿9,192.00",6.38%,2
14,WHAIR,"50,000",฿8.70,"฿435,000.00",฿0.5461,"฿27,305.00",6.28%,2
24,RCL,"24,000",฿40.15,"฿963,600.00",฿2.5000,"฿60,000.00",6.23%,3
7,WHART,"25,000",฿11.60,"฿290,000.00",฿0.6920,"฿17,300.00",5.97%,2
18,NER,"27,000",฿7.45,"฿201,150.00",฿0.3600,"฿9,720.00",4.83%,2


In [12]:
buy[buy['cost_pct'] >= 5.00].shape

(9, 8)

In [13]:
buy[buy['cost_pct'] >= 5.00].div_amt.sum()

236313.1

In [14]:
buy[buy['cost_pct'] < 5.00].shape

(19, 8)

In [15]:
buy[buy['cost_pct'] < 5.00].div_amt.sum()

168854.5

In [16]:
file_name = 'hi-dividend.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [17]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\hi-dividend.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\hi-dividend.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\hi-dividend.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\hi-dividend.csv


In [18]:
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(output_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(god_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(icd_file, index=False)
buy[colu].sort_values(['cost_pct'],ascending=[False]).to_csv(osd_file, index=False)

### Start of Period Calculation

In [20]:
sql = '''
SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B JOIN price P
ON B.name = P.name
WHERE P.date = "%s"
AND active = 1'''
sql = sql % yesterday 
print(sql)


SELECT B.name, volbuy, B.price AS unit_cost, 
dividend, P.price AS mkt_price, period
FROM buy B JOIN price P
ON B.name = P.name
WHERE P.date = "2025-08-08"
AND active = 1


In [21]:
df = pd.read_sql(sql, const)
df.rename(columns={'volbuy':'shares'},inplace=True)
df['shares'] = df.shares.astype(int)
df['cost_amt'] = round(df['shares'] * df['unit_cost'],2)
df['market_amt'] = round(df['shares'] * df['mkt_price'],2)
df['div_amt'] = round(df['shares'] * df['dividend'],2)
df['cost_pct'] = round(df['dividend'] / df['unit_cost'] * 100,2)
df['mkt_pct'] = round(df['dividend'] / df['mkt_price'] * 100,2)
df['profit'] = round((df['mkt_price'] - df['unit_cost']) * df['shares'],2)
df['pft_pct'] = round(df['profit'] / df['cost_amt'] * 100,2)
df[colv].sort_values(['pft_pct'],ascending=[False]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
15,PTT,"2,500",฿32.00,"฿80,000.00",฿32.00,"฿80,000.00",฿2.1000,"฿5,250.00",6.56%,6.56%,฿0.00,0.00%
25,TVO,"8,000",฿24.00,"฿192,000.00",฿23.60,"฿188,800.00",฿1.7300,"฿13,840.00",7.21%,7.33%,"฿-3,200.00",-1.67%
1,AIMIRT,"15,000",฿10.75,"฿161,250.00",฿9.90,"฿148,500.00",฿0.8500,"฿12,750.00",7.91%,8.59%,"฿-12,750.00",-7.91%
6,GVREIT,"69,000",฿7.75,"฿534,750.00",฿6.65,"฿458,850.00",฿0.7989,"฿55,124.10",10.31%,12.01%,"฿-75,900.00",-14.19%
23,TFFIF,"20,000",฿7.20,"฿144,000.00",฿6.05,"฿121,000.00",฿0.4596,"฿9,192.00",6.38%,7.60%,"฿-23,000.00",-15.97%
27,WHART,"25,000",฿11.60,"฿290,000.00",฿9.30,"฿232,500.00",฿0.6920,"฿17,300.00",5.97%,7.44%,"฿-57,500.00",-19.83%
17,RCL,"24,000",฿40.15,"฿963,600.00",฿31.25,"฿750,000.00",฿2.5000,"฿60,000.00",6.23%,8.00%,"฿-213,600.00",-22.17%
3,BCH,"4,000",฿21.70,"฿86,800.00",฿14.30,"฿57,200.00",฿0.4000,"฿1,600.00",1.84%,2.80%,"฿-29,600.00",-34.10%
5,DIF,"40,000",฿13.10,"฿524,000.00",฿8.45,"฿338,000.00",฿0.8888,"฿35,552.00",6.78%,10.52%,"฿-186,000.00",-35.50%
4,CPNREIT,"55,000",฿18.00,"฿990,000.00",฿11.50,"฿632,500.00",฿0.5705,"฿31,377.50",3.17%,4.96%,"฿-357,500.00",-36.11%


In [22]:
df.nlargest(5, 'mkt_pct')[colw].style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct,pft_pct
6,GVREIT,"฿534,750.00","฿458,850.00","฿55,124.10",10.31%,12.01%,-14.19%
5,DIF,"฿524,000.00","฿338,000.00","฿35,552.00",6.78%,10.52%,-35.50%
26,WHAIR,"฿435,000.00","฿275,000.00","฿27,305.00",6.28%,9.93%,-36.78%
1,AIMIRT,"฿161,250.00","฿148,500.00","฿12,750.00",7.91%,8.59%,-7.91%
11,MCS,"฿1,198,800.00","฿700,650.00","฿56,700.00",4.73%,8.09%,-41.55%


In [23]:
df.nsmallest(5, 'mkt_pct')[colw].style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct,pft_pct
20,SINGER,"฿148,800.00","฿34,200.00",฿0.00,0.00%,0.00%,-77.02%
13,ORI,"฿450,000.00","฿106,000.00","฿1,050.00",0.23%,0.99%,-76.44%
16,PTTGC,"฿388,500.00","฿144,600.00","฿3,000.00",0.77%,2.07%,-62.78%
8,3BBIF,"฿1,212,000.00","฿762,000.00","฿18,552.00",1.53%,2.43%,-37.13%
18,SCC,"฿243,000.00","฿121,800.00","฿3,000.00",1.23%,2.46%,-49.88%


In [24]:
file_name = 'buy-div-price.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [25]:
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(output_file)
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(god_file)
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(icd_file)
df[colx].sort_values(['period','name'],ascending=[True,True]).to_csv(osd_file)

### Short term stocks

In [27]:
short_term = df.period == '4'
df.loc[short_term][colv].sort_values(['pft_pct'],ascending=[False]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
14,PTG,"3,600",฿11.40,"฿41,040.00",฿6.75,"฿24,300.00",฿0.3500,"฿1,260.00",3.07%,5.19%,"฿-16,740.00",-40.79%
24,TOA,"1,000",฿26.00,"฿26,000.00",฿14.60,"฿14,600.00",฿0.6000,฿600.00,2.31%,4.11%,"฿-11,400.00",-43.85%
10,JMT,"4,200",฿55.50,"฿233,100.00",฿12.00,"฿50,400.00",฿0.6600,"฿2,772.00",1.19%,5.50%,"฿-182,700.00",-78.38%


In [28]:
p4cost = df.loc[short_term].cost_amt.sum()
p4profit = df.loc[short_term].profit.sum()
p4pct = round(p4profit/p4cost*100, 2)
p4cost, p4profit, p4pct, df.loc[short_term].shape[0]

(300140.0, -210840.0, -70.25, 3)

In [29]:
array = pd.Series([p4cost, p4profit, p4pct])
array = array.map('฿{:,.2f}'.format)
for value in array:
    print(f"The value is: {value}")

The value is: ฿300,140.00
The value is: ฿-210,840.00
The value is: ฿-70.25


### Long term stocks

In [31]:
long_term = df.period == '3'
df[long_term].sort_values(['pft_pct'],ascending=[False]).shape

(7, 13)

In [32]:
df[long_term].nlargest(5, 'pft_pct')[['name','cost_amt','market_amt','profit','pft_pct']].style.format(format_dict)

,name,cost_amt,market_amt,profit,pft_pct
25,TVO,"฿192,000.00","฿188,800.00","฿-3,200.00",-1.67%
17,RCL,"฿963,600.00","฿750,000.00","฿-213,600.00",-22.17%
3,BCH,"฿86,800.00","฿57,200.00","฿-29,600.00",-34.10%
8,3BBIF,"฿1,212,000.00","฿762,000.00","฿-450,000.00",-37.13%
7,IVL,"฿288,000.00","฿164,160.00","฿-123,840.00",-43.00%


In [33]:
p3cost = df.loc[long_term].cost_amt.sum()
p3profit = df.loc[long_term].profit.sum()
p3pct = round(p3profit/p3cost*100, 2)
p3cost, p3profit, p3pct,df.loc[long_term].shape[0]

(3504400.0, -1225490.0, -34.97, 7)

In [34]:
array = pd.Series([p3cost, p3profit, p3pct])
array = array.map('฿{:,.2f}'.format)
for value in array:
    print(f"The value is: {value}")

The value is: ฿3,504,400.00
The value is: ฿-1,225,490.00
The value is: ฿-34.97


### High dividend stocks

In [36]:
hi_dividend = df.period == '2'
df.loc[hi_dividend][colv].sort_values(['mkt_pct'],ascending=[False]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
6,GVREIT,"69,000",฿7.75,"฿534,750.00",฿6.65,"฿458,850.00",฿0.7989,"฿55,124.10",10.31%,12.01%,"฿-75,900.00",-14.19%
5,DIF,"40,000",฿13.10,"฿524,000.00",฿8.45,"฿338,000.00",฿0.8888,"฿35,552.00",6.78%,10.52%,"฿-186,000.00",-35.50%
26,WHAIR,"50,000",฿8.70,"฿435,000.00",฿5.50,"฿275,000.00",฿0.5461,"฿27,305.00",6.28%,9.93%,"฿-160,000.00",-36.78%
1,AIMIRT,"15,000",฿10.75,"฿161,250.00",฿9.90,"฿148,500.00",฿0.8500,"฿12,750.00",7.91%,8.59%,"฿-12,750.00",-7.91%
11,MCS,"81,000",฿14.80,"฿1,198,800.00",฿8.65,"฿700,650.00",฿0.7000,"฿56,700.00",4.73%,8.09%,"฿-498,150.00",-41.55%
12,NER,"27,000",฿7.45,"฿201,150.00",฿4.58,"฿123,660.00",฿0.3600,"฿9,720.00",4.83%,7.86%,"฿-77,490.00",-38.52%
23,TFFIF,"20,000",฿7.20,"฿144,000.00",฿6.05,"฿121,000.00",฿0.4596,"฿9,192.00",6.38%,7.60%,"฿-23,000.00",-15.97%
27,WHART,"25,000",฿11.60,"฿290,000.00",฿9.30,"฿232,500.00",฿0.6920,"฿17,300.00",5.97%,7.44%,"฿-57,500.00",-19.83%
15,PTT,"2,500",฿32.00,"฿80,000.00",฿32.00,"฿80,000.00",฿2.1000,"฿5,250.00",6.56%,6.56%,฿0.00,0.00%
19,SENA,"105,000",฿4.48,"฿470,400.00",฿1.78,"฿186,900.00",฿0.1112,"฿11,676.00",2.48%,6.25%,"฿-283,500.00",-60.27%


In [37]:
df[hi_dividend].nlargest(10, 'mkt_pct')[['name','cost_amt','market_amt','div_amt','cost_pct','mkt_pct']]\
.style.format(format_dict)

,name,cost_amt,market_amt,div_amt,cost_pct,mkt_pct
6,GVREIT,"฿534,750.00","฿458,850.00","฿55,124.10",10.31%,12.01%
5,DIF,"฿524,000.00","฿338,000.00","฿35,552.00",6.78%,10.52%
26,WHAIR,"฿435,000.00","฿275,000.00","฿27,305.00",6.28%,9.93%
1,AIMIRT,"฿161,250.00","฿148,500.00","฿12,750.00",7.91%,8.59%
11,MCS,"฿1,198,800.00","฿700,650.00","฿56,700.00",4.73%,8.09%
12,NER,"฿201,150.00","฿123,660.00","฿9,720.00",4.83%,7.86%
23,TFFIF,"฿144,000.00","฿121,000.00","฿9,192.00",6.38%,7.60%
27,WHART,"฿290,000.00","฿232,500.00","฿17,300.00",5.97%,7.44%
15,PTT,"฿80,000.00","฿80,000.00","฿5,250.00",6.56%,6.56%
19,SENA,"฿470,400.00","฿186,900.00","฿11,676.00",2.48%,6.25%


In [38]:
p2profit = df.loc[hi_dividend].profit.sum()
p2cost = df.loc[hi_dividend].cost_amt.sum()
p2dividend = df.loc[hi_dividend].div_amt.sum()
p2yield = round(p2profit/p2cost*100,2)
p2cost, p2profit, p2yield, p2dividend, df.loc[hi_dividend].shape[0]

(5029350.0, -1731790.0, -34.43, 271946.6, 11)

In [39]:
array = pd.Series([p2cost, p2profit, p2yield, p2dividend])
array = array.map('฿{:,.2f}'.format)
for value in array:
    print(f"The value is: {value}")

The value is: ฿5,029,350.00
The value is: ฿-1,731,790.00
The value is: ฿-34.43
The value is: ฿271,946.60


### Disposal stocks

In [41]:
disposal = df.period == '1'
df.loc[disposal][colv].sort_values(['pft_pct'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,mkt_price,market_amt,dividend,div_amt,cost_pct,mkt_pct,profit,pft_pct
20,SINGER,"6,000",฿24.80,"฿148,800.00",฿5.70,"฿34,200.00",฿0.0000,฿0.00,0.00%,0.00%,"฿-114,600.00",-77.02%
13,ORI,"50,000",฿9.00,"฿450,000.00",฿2.12,"฿106,000.00",฿0.0210,"฿1,050.00",0.23%,0.99%,"฿-344,000.00",-76.44%
9,JMART,"6,800",฿33.00,"฿224,400.00",฿9.35,"฿63,580.00",฿0.4700,"฿3,196.00",1.42%,5.03%,"฿-160,820.00",-71.67%
16,PTTGC,"6,000",฿64.75,"฿388,500.00",฿24.10,"฿144,600.00",฿0.5000,"฿3,000.00",0.77%,2.07%,"฿-243,900.00",-62.78%
0,AH,"1,200",฿37.00,"฿44,400.00",฿15.60,"฿18,720.00",฿0.7800,฿936.00,2.11%,5.00%,"฿-25,680.00",-57.84%
2,AWC,"9,000",฿4.96,"฿44,640.00",฿2.26,"฿20,340.00",฿0.0750,฿675.00,1.51%,3.32%,"฿-24,300.00",-54.44%
18,SCC,600,฿405.00,"฿243,000.00",฿203.00,"฿121,800.00",฿5.0000,"฿3,000.00",1.23%,2.46%,"฿-121,200.00",-49.88%


In [42]:
p1cost = df.loc[disposal].cost_amt.sum()
p1profit = df.loc[disposal].profit.sum()
p1pct = round(p1profit/p1cost*100,2)
p1cost, p1profit, p1pct,df.loc[disposal].shape[0]

(1543740.0, -1034500.0, -67.01, 7)

In [43]:
array = pd.Series([p1cost, p1profit, p1pct])
array = array.map('฿{:,.2f}'.format)
for value in array:
    print(f"The value is: {value}")

The value is: ฿1,543,740.00
The value is: ฿-1,034,500.00
The value is: ฿-67.01


In [44]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y:%m:%d %H:%M:%S")
print(formatted_time)

2025:08:12 20:59:49
